# Setup ice box
- get ice config with https://github.com/vitroid/GenIce
- equilibrate with anisotropic barostat (optional)
- equilibrate with fixed volume
- run MD at fixed volume

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import clear_output
from tqdm.auto import trange

In [ ]:
import openmm, openmm.app
from openmm import unit
kB = unit.MOLAR_GAS_CONSTANT_R.value_in_unit(unit.kilojoule_per_mole/unit.kelvin)

In [ ]:
from watermodel import *

## generate ice configuration and setup water model
see https://github.com/vitroid/GenIce

In [ ]:
#some options

temp = 250
ice_type = 'Ih'
rep = 3*[1]

nonbondedCutoff = 1
external_field = None

anisotropic_equilibration = False

# !genice2 -h  #see available ice_types

In [ ]:
#setup the model
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule
from tempfile import NamedTemporaryFile

gro = GenIce(Lattice(ice_type), rep=rep).generate_ice(Format('gromacs'), water=Molecule('tip4p'))
tmp = NamedTemporaryFile()
with open(tmp.name, 'w') as f:
    f.write(gro)
config = openmm.app.GromacsGroFile(tmp.name)

pos = np.array(config.getPositions().value_in_unit(unit.nanometer))
box = np.array(config.getPeriodicBoxVectors().value_in_unit(unit.nanometer))
model = WaterModel(pos, box, nonbondedCutoff=nonbondedCutoff, external_field=external_field)

print('n_waters:', model.n_waters)
print('orthorombic:', model.is_box_orthorombic)
print(f'nonbondedCutoff: {model.nonbondedCutoff} ({model.nonbondedCutoff/0.316435:g} [sigmaLJ])') #3.16 is common, 2.5 is ok, below 1.14 is very bad

model.get_view()

## anisotropic equilibration

In [ ]:
%%time 
#equilibrate
if anisotropic_equilibration:
    model.set_barostat('aniso')
    print('barostat:', model.barostat)

    pace = 500
    n_iter = 10_000   
    simulation = setup_simulation(model, temp, minimizeEnergy=True)
    
    MDene = np.full(n_iter, np.nan)
    MDpos = np.full((n_iter, *model.positions.shape), np.nan)
    if model.barostat is None:
        MDbox = np.resize(model.box, (1,3,3))
    else:
        MDbox = np.full((n_iter, 3, 3), np.nan)

    for n in trange(n_iter):
        simulation.step(pace)
        MDene[n] = simulation.context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
        MDpos[n] = simulation.context.getState(getPositions=True, enforcePeriodicBox=True).getPositions(asNumpy=True).value_in_unit(unit.nanometers)
        if model.barostat is not None:
            MDbox[n] = simulation.context.getState().getPeriodicBoxVectors(asNumpy=True).value_in_unit(unit.nanometers)

In [ ]:
#visualize
if anisotropic_equilibration:
    for i in range(3):
        plt.plot(MDbox[:,i,i], '.')
        print(box[i,i], MDbox[:,i,i].mean(), MDbox[-len(MDbox)//2:,i,i].mean())
        plt.axhline(MDbox[-len(MDbox)//2:,i,i].mean(), c='k', ls='--')
        plt.axhline(model.box[i,i], c='k', ls=':')
        # plt.ylim(0,None)
    plt.show()
    
    plot_energy(MDene)
    plot_2Dview(MDpos, MDbox)
    # model.get_view(MDpos, MDbox)

In [ ]:
#update the model
if anisotropic_equilibration:
    model.positions = MDpos[-1]
    model.box = MDbox[-len(MDbox)//2:].mean(axis=0)
    model.system.getForces()

## fixed volume equilibration

In [ ]:
%%time
#Equilibrate

model.set_barostat = None
print('barostat:', model.barostat)

pace = 500
n_iter = 10_000
simulation = setup_simulation(model, temp, minimizeEnergy=True)

MDene = np.full(n_iter, np.nan)
MDpos = np.full((n_iter, *model.positions.shape), np.nan)
if model.barostat is None:
    MDbox = np.resize(model.box, (1,3,3))
else:
    MDbox = np.full((n_iter, 3, 3), np.nan)

for n in trange(n_iter):
    simulation.step(pace)
    MDene[n] = simulation.context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
    MDpos[n] = simulation.context.getState(getPositions=True, enforcePeriodicBox=True).getPositions(asNumpy=True).value_in_unit(unit.nanometers)
    if model.barostat is not None:
        MDbox[n] = simulation.context.getState().getPeriodicBoxVectors(asNumpy=True).value_in_unit(unit.nanometers)

In [ ]:
#visualize
plot_energy(MDene)
plot_2Dview(MDpos, MDbox)
# model.get_view(MDpos, MDbox)

In [ ]:
#update model
model.positions = MDpos[-1]
model.box = MDbox[-1]
model.system.getForces()

In [ ]:
# raise SystemError('Safety break')

## run MD storing model and trajectory

In [ ]:
%%time
#production run

print('barostat:', model.barostat)

pace = 500
n_iter = 100_000
simulation = setup_simulation(model, temp)

MDene = np.full(n_iter, np.nan)
MDpos = np.full((n_iter, *model.positions.shape), np.nan)
if model.barostat is None:
    MDbox = np.resize(model.box, (1,3,3))
else:
    MDbox = np.full((n_iter, 3, 3), np.nan)

for n in trange(n_iter):
    simulation.step(pace)
    MDene[n] = simulation.context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
    MDpos[n] = simulation.context.getState(getPositions=True, enforcePeriodicBox=True).getPositions(asNumpy=True).value_in_unit(unit.nanometers)
    if model.barostat is not None:
        MDbox[n] = simulation.context.getState().getPeriodicBoxVectors(asNumpy=True).value_in_unit(unit.nanometers)

In [ ]:
#visualize
plot_energy(MDene)
plot_2Dview(MDpos[::10], MDbox[::10])
# model.get_view(MDpos, MDbox)

In [ ]:
# raise SystemError('Safety break')

In [ ]:
#save trajectory
#NB: positions can be out of PBC, to avoid breaking molecules
info = f'ice{ice_type}_T{temp}_N{model.n_waters}'
filename = f'model-{info}.json'
!bck.meup.sh -v {filename}
model.save_to_json(filename)

filename = f'MDtraj-{info}.npz'
!bck.meup.sh -v {filename}
np.savez(filename, pos=MDpos, box=MDbox, ene=MDene)

#save also to scratch
scratch_folder = f'/group/ag_cmb/scratch/minvernizzi/so3-flow/ice_MDdata/'
!bck.meup.sh -v {scratch_folder}*{info}*; cp *{info}* {scratch_folder}

In [ ]:
# #load model from file
# filename = f'model-{info}.json'
# new_model = WaterModel.load_from_json(filename)